In [1]:
import numpy as np
import pandas as pd


import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *



#################################             FUNCTIONNALITY             ########################################


# this programs reads a csv file from the TBF110. A year is required to launch this program

# From the year selected, the program counts the sum ITEMS_TOTAL_AMOUNT for each sender_id and each month

# It creates 0 values for the non existing dates for each sender_id and stores it in a csv for later visualizing


#################################################################################################################



# Choose your year and your month

yearChosen = 2017


# Set up the spark session

spark = SparkSession.builder \
   .master("spark://spark-master:7077") \
   .appName("sumbymonth") \
   .config(conf = pyspark.SparkConf()) \
   .getOrCreate()


# Read the csv File

df1 = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('../sparkJup/romain.csv')




dftest = df1.filter(df1['year'] == yearChosen).groupby("month","SENDER_SITE_LOC_ID").agg(sum('ITEMS_TOTAL_AMOUNT'))
dftest.show()


# Transforms to Pandas Dataframe

dfMonth = dftest.toPandas()

i = 0

# Remove the duplicates for sender_id

senderId = pd.unique(dfMonth['SENDER_SITE_LOC_ID'].values)

# A table with all the dates which occur one time

dateByMonth = [t for t in range(1,13)]


# Set up the table length

lenDateByMonth = len(dateByMonth)
lenSenderId = len(senderId)

print(senderId)
print(dateByMonth)


#print(lenDateByYear)
#print(lenSenderId)


# This loop is used to create some rows for sender_id which did not have all the dates
# Example sender_id = 1 month = 1,8 dateByMonth=[1,2,3,..,10,11,12]
# A new row is created to have sender_id = 1 month = 2 and sender_id = 1 month = 3 etc
# This is compulsory for creating visualization

while i < lenSenderId:
    for j in range(lenDateByMonth):
        if i+j < lenSenderId + lenDateByMonth:
            dfBoolean = dfMonth.loc[dfMonth['SENDER_SITE_LOC_ID'].isin([senderId[i]]) & dfMonth['month'].isin([dateByMonth[j]])]
            #print(senderId[i])
            #print(dateByYear[j])
            #print(dftemp)
            if (dfBoolean.empty == True):
                df3 = pd.DataFrame(np.array([[dateByMonth[j],senderId[i]]]), columns=['month','SENDER_SITE_LOC_ID'])
                dfMonth = dfMonth.append(df3, ignore_index=True)
#                #print(dfsum['SENDER_SITE_LOC_ID'][i])
#                #print(dfsum['year'][i])
#                print(tab2[j])
#                print('not ok')
            else :
                print('ok')
                #print(dfsum['SENDER_SITE_LOC_ID'][i])
                #print(dfsum['year'][i])
                #print(tab2[j])


        else:
            if(dfsum['month'][i+j - 1] != dateByMonth[len(dateByMonth)-1]):
                #print('make something')
                df3 = pd.DataFrame(np.array([[dateByMonth[j],senderId[i]]]), columns=['month','SENDER_SITE_LOC_ID'])
                dfMonth = dfMonth.append(df3, ignore_index=True)
    i = i + 1


print(dfMonth)


# Replace the values which are NaN by 0
dfMonth= dfMonth.fillna(0)

#print(dfsum)

# Save to a csv file
dfMonth.to_csv('sumbymonth.csv')

spark.stop()



+-----+------------------+-----------------------+
|month|SENDER_SITE_LOC_ID|sum(ITEMS_TOTAL_AMOUNT)|
+-----+------------------+-----------------------+
|    7|             15801|            12531848256|
|    7|              7597|              524884448|
|    7|              8627|            28521692304|
|    5|             15793|           697887644986|
|    8|             15705|             1059237974|
|    7|             15695|               12979272|
|    7|             15793|            51087133316|
+-----+------------------+-----------------------+

[15801  7597  8627 15793 15705 15695]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
ok
ok
ok
ok
ok
ok
ok
    SENDER_SITE_LOC_ID  month  sum(ITEMS_TOTAL_AMOUNT)
0                15801      7             1.253185e+10
1                 7597      7             5.248844e+08
2                 8627      7             2.852169e+10
3                15793      5             6.978876e+11
4                15705      8             1.059238e+09
5        